# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.69,69,3,10.16,SH,1697423391
1,1,port alfred,-33.5906,26.8910,17.32,93,100,2.24,ZA,1697423391
2,2,simanggang,1.2472,111.4528,30.99,70,20,1.54,MY,1697423391
3,3,corn island,12.1694,-83.0418,29.52,72,14,2.75,NI,1697423391
4,4,ust-nera,64.5667,143.2000,-13.21,95,50,2.68,RU,1697423391


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_data_df['Lng']

0      -12.3116
1       26.8910
2      111.4528
3      -83.0418
4      143.2000
         ...   
619     83.5500
620    -55.9833
621    -16.7667
622    -90.2992
623     20.2263
Name: Lng, Length: 624, dtype: float64

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5
)
# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria - temp between 20 and 25 celsius and no clouds
sunny_cities = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
sunny_cities = sunny_cities.dropna()

# Display sample data
sunny_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,kamaishi,39.2667,141.8833,20.33,54,0,6.61,JP,1697423406
123,123,xiaoweizhai,26.1903,107.5125,22.99,54,0,1.11,CN,1697423409
163,163,siwa oasis,29.2041,25.5195,21.04,46,0,2.15,EG,1697423416
195,195,tindouf,27.6711,-8.1474,24.25,22,0,2.06,DZ,1697423421
245,245,hasaki,35.7333,140.8333,22.22,53,0,2.45,JP,1697423428
265,265,tsuma,32.1000,131.4000,23.97,46,0,0.70,JP,1697423430
279,279,duyun,26.2667,107.5167,22.51,56,0,1.21,CN,1697423432
330,330,al kharijah,25.4514,30.5464,20.35,41,0,3.42,EG,1697423440
338,338,san patricio,28.0170,-97.5169,22.92,40,0,3.98,US,1697423441
365,365,shingu,33.7333,135.9833,23.54,41,0,2.83,JP,1697423445


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sunny_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,kamaishi,JP,39.2667,141.8833,54,
123,xiaoweizhai,CN,26.1903,107.5125,54,
163,siwa oasis,EG,29.2041,25.5195,46,
195,tindouf,DZ,27.6711,-8.1474,22,
245,hasaki,JP,35.7333,140.8333,53,
265,tsuma,JP,32.1000,131.4000,46,
279,duyun,CN,26.2667,107.5167,56,
330,al kharijah,EG,25.4514,30.5464,41,
338,san patricio,US,28.0170,-97.5169,40,
365,shingu,JP,33.7333,135.9833,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)